# AGYW_PREV DATIM

In [1]:
# load the modules
%run layeringOfServices.py

In [2]:
dreams_mastersheet.dtypes

main_id                     int64
muso_eligibility           object
actual_id_group             int64
group_she_take_sessions    object
code                       object
                            ...  
recevoir_1services         object
ps_10_14                   object
ps_15_19                   object
ps_20_24                   object
score_eligible_AGYW        object
Length: 283, dtype: object

## Datim Pour les 3 definitions